Imports

In [ ]:
import multiprocessing as mp
import random
from models import Order, SimulationEnvironment

Classes #TODO: outsource to py file and import into this one

In [3]:
class Agent(): # agent base class, setup for multiprocessing
    def __init__(self, result_size : int):
        # write results of each agent to this variable
        self.RESULT = mp.Array('f', result_size, True) #Note: f - indicates use of float values, True = Lock Object for synchronization is created
        
    def configure(self):
        pass

    def write_result(self, result):
        with self.RESULT.get_lock():
            self.RESULT = result

    def run():
        pass

class GreedyAgent(Agent):

    def confiugure(self, environment : SimulationEnvironment):
        self.environment = environment

    def pick_random(self, task, tasks):
        task = self.environment.get_task(task)
        if len(task.follow_up_tasks) > 0:
            follow_up_id = random.choice(task.follow_up_tasks)
            follow_up = self.environment.get_task(follow_up_id)
            tasks.append(follow_up)
            self.pick_random(follow_up.id, tasks)
        
    def pick_random_path(self, recipe):
        tasks = []
        task = random.choice(recipe.tasks)
        tasks.append(task)
        self.pick_random(task.id, tasks)
        return tasks 
    
    def assignments_for(self, id):
        assignments = []
        for i in range(0, len(self.RESULT), 4):
            if self.RESULT[i] == id:
                assignments.push_back(self.RESULT[i], self.RESULT[i+1], self.RESULT[i+2], self.RESULT[i+3])
        return assignments

    def is_blocked(self, workstation_id, start_time, duration):
        assignments = self.assignments_for(workstation_id)
        if assignments:
            for assignment in assignments:
                assignment_start_time = assignment[1]
                assignment_duration = self.environment.get_duration(assignment[0], workstation_id)
                if start_time > assignment_start_time and start_time < assignment_start_time + assignment_duration:
                    return True
                if start_time + duration > assignment_start_time and start_time + duration < assignment_start_time + assignment_duration:
                    return True
                # if new task is longer than already scheduled task, it could start before and end after -> overlap
                if assignment_start_time > start_time and assignment_start_time < start_time + duration:
                    return True
                if assignment_start_time + assignment_duration > start_time and assignment_start_time + assignment_duration < start_time + duration:
                    return True
        return False

    def run(self, order : Order):
        recipes = []
        for resource in order.resources:
            recipes.append((order.id, resource.recipes))
        tasks = []
        for recipe in recipes:
            path = recipe[1][0].tasks#self.pick_random_path(recipe[1][0]) # ignore possibility of multiple recipes for now
            path.reverse()
            tasks.append(path) # add in reverse order for each recipe
        success_count = 0
        for i in range(len(tasks)):
            prev_duration = 0
            for j in range(len(tasks[i])):
                task = tasks[i][j]
                workstations = self.environment.get_valid_workstations(task.external_id)
                chosen_workstation = None
                for workstation in workstations:
                    duration = self.environment.get_duration(task.external_id, workstation.external_id)
                    start_slot = order.delivery_time - prev_duration - duration
                    if not self.is_blocked(workstation.external_id, start_slot, duration):
                        chosen_workstation = workstation
                        break            
                if chosen_workstation:    
                    prev_duration += duration
                    self.RESULT.push_back(chosen_workstation)
                    self.RESULT.push_back(start_slot)
                    self.RESULT.push_back(task.external_id)
                    self.RESULT.push_back(order.external_id)
                    success_count += 1
                else:
                    if not order.divisible: # if order can not only be partially fullfilled too
                        return False
                    else:
                        break
        return success_count > 0

Helper Functions

In [ ]:
def create_agent(type): # TODO: add problem information
    pass

Setups

In [ ]:
agents = []
# for homogenous systems
n = 10
types = []
for i in range(n):
    types.append(0)

# for heterogenous systems
types = [0, 1, 1, 2, 0, 3, 3, 4, 1, 0] # example


for type in types:
    agents.append(create_agent(type)) # add n agents

Run

In [ ]:
results = []
processes = []
for agent in agents:
    process = mp.Process(target=agent.run(), args=())
    process.start()
    processes.append(process)
    
for i in range(len(processes)):
    processes[i].join()
    results.append(agents[i].RESULT)
